<a href="https://colab.research.google.com/github/Nathan-Nicolau/RedesNeuraisTCC/blob/main/Rede_Vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Substitua 'caminho_para_sua_pasta_principal_com_24_subpastas' pelo caminho real no seu Google Drive
base_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/CromossomosAjustados_100_x_100/'

Mounted at /content/drive


In [ ]:
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Substitua 'caminho_para_sua_pasta_principal_com_24_subpastas' pelo caminho real
base_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/CromossomosAjustados_100_x_100/'

# Use a função ImageDataGenerator para aplicar aumentação de dados durante o treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Divida as pastas em treinamento e teste
train_dir = os.path.join(base_dir)
test_dir = os.path.join(base_dir)

# Carregue dados de treinamento e teste usando o ImageDataGenerator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 100),
    batch_size=256,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 100),
    batch_size=256,
    class_mode='categorical'
)

# Crie o modelo VGG16
base_model = VGG16(
    weights='imagenet', include_top=False, input_shape=(150, 100, 3))

# Congele as camadas convolucionais do modelo VGG16 pré-treinado
for layer in base_model.layers:
    layer.trainable = False

# Adicione camadas personalizadas no topo
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(24, activation='softmax'))  # 24 classes de saída

# Compile o modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy',
              metrics=['accuracy'])

# Registre o tempo de início
start_time = time.time()

# Treine o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Registre o tempo de término
end_time = time.time()

# Calcule o tempo total de execução
total_time = end_time - start_time

print(f'Tempo total de execução: {total_time:.2f} segundos')

# Avalie o modelo
test_loss, test_acc = model.evaluate(
    test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc}')

# Visualize o desempenho do treinamento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

# Gráfico de Barras para Métricas Numéricas
y_true = test_generator.classes
y_scores = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size).argmax(axis=1)

accuracy = accuracy_score(y_true, y_scores)
precision = precision_score(y_true, y_scores, average='weighted')
recall = recall_score(y_true, y_scores, average='weighted')
f1 = f1_score(y_true, y_scores, average='weighted')

metrics = ['Acurácia', 'Precisão', 'Recall', 'Pontuação F1']
values = [accuracy, precision, recall, f1]

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.bar(metrics, values, color=['blue', 'green', 'orange', 'red'])
plt.ylabel('Valor')
plt.title('Métricas de Desempenho')

# Matriz de Confusão
plt.subplot(1, 3, 2)
confusion_mat = confusion_matrix(y_true, y_scores)
sns.heatmap(confusion_mat, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')

# Gráfico ROC
plt.subplot(1, 3, 3)
fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')

plt.show()

# Salvar o modelo treinado
model.save('RedeVGG16_treinada.h5')

Found 12013 images belonging to 24 classes.
Found 12013 images belonging to 24 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/100


In [ ]:
#Uso do modelo treinado para reconhecer novas imagens
from google.colab import drive
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Monte o Google Drive
drive.mount('/content/drive')

# Carregue o modelo treinado
loaded_model = load_model('/content/drive/MyDrive/seu_caminho/RedeVGG16_treinada.h5')

# Substitua 'caminho_para_pastas_principais_com_24_subpastas' pelo caminho real no Google Drive
base_folder = '/content/drive/MyDrive/seu_caminho/NovasImagens/'

# Loop através das 24 pastas
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    # Loop através das imagens em cada pasta
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        # Carregue e pré-processe a imagem
        new_image = image.load_img(image_path, target_size=(150, 100))
        new_image = image.img_to_array(new_image)
        new_image = np.expand_dims(new_image, axis=0)
        new_image = new_image / 255.0  # Normalização

        # Faça a previsão
        predictions = loaded_model.predict(new_image)

        # Exiba as previsões ou faça o que for necessário com elas
        print(f'Pasta: {folder_name}, Imagem: {image_name}, Previsões: {predictions}')
